In [ ]:
print("I am fine")

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("/120040051/hf_llama2")
model = LlamaForCausalLM.from_pretrained("/120040051/hf_llama2").to(device)

In [ ]:
def calculate_perplexity(sentence, tokenizer, model, device):
    model.eval()  # Ensure the model is in evaluation mode
    # Tokenize the input sentence
    encodings = tokenizer(sentence, return_tensors="pt")

    # Move encodings to the same device as the model
    input_ids = encodings.input_ids.to(device)
    target_ids = input_ids.clone()
    attention_mask = encodings.attention_mask.to(device)

    # Calculate Negative Log-Likelihood
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids, attention_mask=attention_mask)
        neg_log_likelihood = outputs.loss * input_ids.size(1)  # Multiply by sequence length

    # Calculate Perplexity and append to list
    ppl = torch.exp(neg_log_likelihood / input_ids.size(1))  # Divide by sequence length
    perplexity = ppl.item()

    return perplexity

### WARNING: High Computing Resources Required

In [ ]:
import json
with open("../sentence_parse/tmp_compositional_sents.json", 'r') as f:
    j_data = json.load(f)
print("Loading Complete")

sent_list = j_data["data"]
sent_list_ppl = []


i = 0
for sent_dict in sent_list:
    sub = sent_dict["subj"]
    if sub in ['A car bomb', 'A doctor', 'A person']: # compromise for now
        text = sent_dict["text"]
        ppl = calculate_perplexity(text, tokenizer, model, device)
        sent_dict["ppl"] = round(ppl, 2)
        sent_list_ppl.append(sent_dict)
        i += 1
    if (i % 1000 == 0):
        print(f"Processed {i} sentences")
print(f"Processed {i} sentences")
j_data["data"] = sent_list_ppl

outfile = "sentence_table_3_subjs.json"

print(f"Writing the results to {outfile}, ...")
with open(outfile, 'w') as f:
    json.dump(j_data, f)
print("Writing complete")

In [ ]:
sent1 = "A man is eating pizza in the restaurant."
perplexity_values = calculate_perplexity(sent1, tokenizer, model, device)
print(perplexity_values)